In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import datetime
import time
import random
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

In [2]:
class SmartACController:
    """
    AI-powered AC temperature controller using machine learning
    """

    def __init__(self):
        self.model = None
        self.scaler = StandardScaler()
        self.is_trained = False
        self.comfort_range = (22, 26)  # Comfortable temperature range
        self.max_temp_change = 2.0     # Maximum temperature change per adjustment
        self.energy_weight = 0.3       # Weight for energy efficiency in decisions

        # Historical data for learning
        self.sensor_data = []
        self.user_adjustments = []

    def generate_training_data(self, num_samples: int = 10000) -> pd.DataFrame:
        """
        Generate synthetic training data to simulate real AC usage patterns
        """
        data = []

        for i in range(num_samples):
            # Time features
            hour = random.randint(0, 23)
            day_of_week = random.randint(0, 6)
            month = random.randint(1, 12)

            # Environmental features
            outdoor_temp = 25 + 10 * np.sin(2 * np.pi * month / 12) + random.gauss(0, 3)

            # Indoor conditions
            base_indoor_temp = outdoor_temp + random.gauss(-2, 2)
            humidity = max(30, min(80, 50 + random.gauss(0, 10)))

            # Occupancy patterns (higher during evening/weekend)
            occupancy_prob = 0.3
            if 18 <= hour <= 23 or day_of_week >= 5:
                occupancy_prob = 0.8
            elif 6 <= hour <= 8 or 12 <= hour <= 14:
                occupancy_prob = 0.6

            occupancy = 1 if random.random() < occupancy_prob else 0

            # Previous AC setting (simulated)
            prev_ac_temp = random.uniform(20, 28)

            # Energy consumption (depends on outdoor temp and AC setting)
            energy_usage = max(0, abs(outdoor_temp - prev_ac_temp) * 0.1 + random.gauss(0, 0.02))

            # Calculate optimal temperature based on comfort and efficiency
            if occupancy:
                # When occupied, prioritize comfort
                if outdoor_temp > 28:
                    optimal_temp = 24 + random.gauss(0, 1)
                elif outdoor_temp < 20:
                    optimal_temp = 23 + random.gauss(0, 1)
                else:
                    optimal_temp = 25 + random.gauss(0, 1)
            else:
                # When unoccupied, prioritize energy saving
                optimal_temp = outdoor_temp + random.gauss(-1, 2)

            # Clamp to reasonable range
            optimal_temp = max(18, min(30, optimal_temp))

            data.append({
                'hour': hour,
                'day_of_week': day_of_week,
                'month': month,
                'outdoor_temp': outdoor_temp,
                'indoor_temp': base_indoor_temp,
                'humidity': humidity,
                'occupancy': occupancy,
                'prev_ac_temp': prev_ac_temp,
                'energy_usage': energy_usage,
                'optimal_temp': optimal_temp
            })

        return pd.DataFrame(data)

    def prepare_features(self, df: pd.DataFrame) -> np.ndarray:
        """
        Prepare feature matrix for ML model
        """
        # Cyclical encoding for time features
        df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
        df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
        df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
        df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

        # Temperature differences
        df['temp_diff'] = df['outdoor_temp'] - df['indoor_temp']
        df['ac_diff'] = df['prev_ac_temp'] - df['indoor_temp']

        # Select features for model
        features = [
            'outdoor_temp', 'indoor_temp', 'humidity', 'occupancy',
            'prev_ac_temp', 'energy_usage', 'temp_diff', 'ac_diff',
            'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos'
        ]

        return df[features].values

    def train_model(self, data: pd.DataFrame = None):
        """
        Train the ML model on historical data
        """
        if data is None:
            print("Generating training data...")
            data = self.generate_training_data()

        # Prepare features and target
        X = self.prepare_features(data)
        y = data['optimal_temp'].values

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Scale features
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        # Train ensemble of models
        models = {
            'random_forest': RandomForestRegressor(n_estimators=100, random_state=42),
            'gradient_boost': GradientBoostingRegressor(n_estimators=100, random_state=42)
        }

        best_score = float('inf')
        best_model = None

        print("Training models...")
        for name, model in models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)

            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)

            print(f"{name} - MSE: {mse:.3f}, MAE: {mae:.3f}")

            if mse < best_score:
                best_score = mse
                best_model = model

        self.model = best_model
        self.is_trained = True
        print(f"Best model selected with MSE: {best_score:.3f}")

        # Feature importance
        if hasattr(self.model, 'feature_importances_'):
            feature_names = [
                'outdoor_temp', 'indoor_temp', 'humidity', 'occupancy',
                'prev_ac_temp', 'energy_usage', 'temp_diff', 'ac_diff',
                'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos'
            ]
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': self.model.feature_importances_
            }).sort_values('importance', ascending=False)

            print("\nFeature Importance:")
            print(importance_df.head(8))

    def get_sensor_data(self) -> Dict:
        """
        Simulate getting current sensor data
        In real implementation, this would read from actual sensors
        """
        now = datetime.datetime.now()

        # Simulate realistic sensor data
        outdoor_temp = 25 + 8 * np.sin(2 * np.pi * now.month / 12) + random.gauss(0, 2)
        indoor_temp = outdoor_temp + random.gauss(-3, 1.5)
        humidity = max(30, min(80, 50 + random.gauss(0, 8)))

        # Simulate occupancy based on time
        occupancy_prob = 0.2
        if 7 <= now.hour <= 9 or 18 <= now.hour <= 23:
            occupancy_prob = 0.9
        elif 10 <= now.hour <= 17:
            occupancy_prob = 0.4

        occupancy = 1 if random.random() < occupancy_prob else 0

        return {
            'hour': now.hour,
            'day_of_week': now.weekday(),
            'month': now.month,
            'outdoor_temp': outdoor_temp,
            'indoor_temp': indoor_temp,
            'humidity': humidity,
            'occupancy': occupancy,
            'prev_ac_temp': getattr(self, 'current_ac_temp', 24),
            'energy_usage': random.uniform(0.5, 2.0)
        }

    def predict_optimal_temperature(self, sensor_data: Dict) -> float:
        """
        Predict optimal AC temperature based on current conditions
        """
        if not self.is_trained:
            raise ValueError("Model must be trained before making predictions")

        # Create feature vector
        df_temp = pd.DataFrame([sensor_data])
        features = self.prepare_features(df_temp)
        features_scaled = self.scaler.transform(features)

        # Get prediction
        predicted_temp = self.model.predict(features_scaled)[0]

        # Apply safety constraints
        current_temp = sensor_data['indoor_temp']
        max_change = self.max_temp_change

        predicted_temp = max(
            current_temp - max_change,
            min(current_temp + max_change, predicted_temp)
        )

        # Ensure within reasonable bounds
        predicted_temp = max(18, min(30, predicted_temp))

        return round(predicted_temp, 1)

    def calculate_comfort_score(self, temp: float, occupancy: int) -> float:
        """
        Calculate comfort score for a given temperature
        """
        if not occupancy:
            return 1.0  # Don't prioritize comfort when unoccupied

        ideal_temp = 24  # Ideal comfort temperature
        deviation = abs(temp - ideal_temp)

        if deviation <= 1:
            return 1.0
        elif deviation <= 2:
            return 0.8
        elif deviation <= 3:
            return 0.5
        else:
            return 0.2

    def control_ac(self, target_temp: float) -> Dict:
        """
        Send control command to AC (simulated)
        In real implementation, this would interface with actual AC unit
        """
        self.current_ac_temp = target_temp

        return {
            'status': 'success',
            'temperature_set': target_temp,
            'timestamp': datetime.datetime.now().isoformat(),
            'mode': 'auto_ml'
        }

    def run_control_cycle(self) -> Dict:
        """
        Complete control cycle: sense -> predict -> act
        """
        # Get current sensor readings
        sensor_data = self.get_sensor_data()

        # Predict optimal temperature
        optimal_temp = self.predict_optimal_temperature(sensor_data)

        # Calculate comfort and efficiency scores
        comfort_score = self.calculate_comfort_score(optimal_temp, sensor_data['occupancy'])

        # Apply control action
        control_result = self.control_ac(optimal_temp)

        # Log the decision
        decision_log = {
            'timestamp': datetime.datetime.now().isoformat(),
            'sensor_data': sensor_data,
            'predicted_temp': optimal_temp,
            'comfort_score': comfort_score,
            'control_result': control_result
        }

        return decision_log

    def adaptive_learning(self, user_adjustment: float, sensor_data: Dict):
        """
        Learn from user manual adjustments to improve future predictions
        """
        # Store user feedback for retraining
        feedback_data = {
            **sensor_data,
            'user_preferred_temp': user_adjustment,
            'timestamp': datetime.datetime.now()
        }

        self.user_adjustments.append(feedback_data)

        # Retrain if we have enough new feedback
        if len(self.user_adjustments) >= 50:
            print("Retraining model with user feedback...")
            self.retrain_with_feedback()

    def retrain_with_feedback(self):
        """
        Retrain model incorporating user feedback
        """
        if len(self.user_adjustments) < 10:
            return

        # Convert user adjustments to training data
        feedback_df = pd.DataFrame(self.user_adjustments)
        feedback_df['optimal_temp'] = feedback_df['user_preferred_temp']

        # Prepare features
        X_feedback = self.prepare_features(feedback_df)
        y_feedback = feedback_df['optimal_temp'].values

        # Scale features
        X_feedback_scaled = self.scaler.transform(X_feedback)

        # Retrain model with new data
        self.model.fit(X_feedback_scaled, y_feedback)

        print("Model retrained with user feedback")
        self.user_adjustments = []  # Clear processed feedback

    def save_model(self, filepath: str):
        """Save trained model and scaler"""
        model_data = {
            'model': self.model,
            'scaler': self.scaler,
            'comfort_range': self.comfort_range,
            'is_trained': self.is_trained
        }
        joblib.dump(model_data, filepath)
        print(f"Model saved to {filepath}")

    def load_model(self, filepath: str):
        """Load pre-trained model and scaler"""
        model_data = joblib.load(filepath)
        self.model = model_data['model']
        self.scaler = model_data['scaler']
        self.comfort_range = model_data['comfort_range']
        self.is_trained = model_data['is_trained']
        print(f"Model loaded from {filepath}")

class RLACController:
    """
    Reinforcement Learning based AC controller using Q-Learning approach
    """

    def __init__(self, learning_rate=0.1, discount_factor=0.95, epsilon=0.1):
        self.lr = learning_rate
        self.gamma = discount_factor
        self.epsilon = epsilon

        # Define action space (temperature adjustments)
        self.actions = [-2, -1, 0, 1, 2]  # Temperature change options
        self.q_table = {}  # Q-value table

        self.current_state = None
        self.current_action = None
        self.comfort_weight = 0.7
        self.energy_weight = 0.3

    def discretize_state(self, sensor_data: Dict) -> Tuple:
        """
        Convert continuous state to discrete state for Q-table
        """
        temp_bucket = int(sensor_data['indoor_temp'] / 2) * 2  # 2-degree buckets
        outdoor_bucket = int(sensor_data['outdoor_temp'] / 5) * 5  # 5-degree buckets
        hour_bucket = sensor_data['hour'] // 4 * 4  # 4-hour buckets

        return (
            temp_bucket,
            outdoor_bucket,
            hour_bucket,
            sensor_data['occupancy'],
            int(sensor_data['humidity'] / 10) * 10  # 10% humidity buckets
        )

    def get_q_value(self, state: Tuple, action: int) -> float:
        """Get Q-value for state-action pair"""
        return self.q_table.get((state, action), 0.0)

    def choose_action(self, state: Tuple) -> int:
        """Choose action using epsilon-greedy policy"""
        if random.random() < self.epsilon:
            return random.choice(self.actions)  # Explore

        # Exploit: choose best action
        q_values = [self.get_q_value(state, action) for action in self.actions]
        best_action_idx = np.argmax(q_values)
        return self.actions[best_action_idx]

    def calculate_reward(self, sensor_data: Dict, action: int, new_temp: float) -> float:
        """
        Calculate reward based on comfort and energy efficiency
        """
        # Comfort reward
        ideal_temp = 24 if sensor_data['occupancy'] else 26
        temp_deviation = abs(new_temp - ideal_temp)
        comfort_reward = max(0, 10 - temp_deviation * 2)

        # Energy penalty (larger changes cost more energy)
        energy_penalty = abs(action) * 0.5

        # Additional penalty for extreme temperatures
        if new_temp < 20 or new_temp > 28:
            energy_penalty += 5

        # When unoccupied, prioritize energy saving
        if not sensor_data['occupancy']:
            energy_penalty *= 2

        total_reward = (
            comfort_reward * self.comfort_weight -
            energy_penalty * self.energy_weight
        )

        return total_reward

    def update_q_table(self, state: Tuple, action: int, reward: float, next_state: Tuple):
        """Update Q-table using Q-learning update rule"""
        current_q = self.get_q_value(state, action)

        # Find maximum Q-value for next state
        next_q_values = [self.get_q_value(next_state, a) for a in self.actions]
        max_next_q = max(next_q_values) if next_q_values else 0

        # Q-learning update
        new_q = current_q + self.lr * (reward + self.gamma * max_next_q - current_q)
        self.q_table[(state, action)] = new_q

    def control_step(self, sensor_data: Dict) -> Dict:
        """
        Perform one control step using RL
        """
        state = self.discretize_state(sensor_data)
        action = self.choose_action(state)

        # Calculate new temperature
        current_ac_temp = sensor_data.get('prev_ac_temp', 24)
        new_temp = max(18, min(30, current_ac_temp + action))

        # Calculate reward
        reward = self.calculate_reward(sensor_data, action, new_temp)

        # Update Q-table if we have previous experience
        if self.current_state is not None and self.current_action is not None:
            self.update_q_table(self.current_state, self.current_action, reward, state)

        # Store current state and action for next update
        self.current_state = state
        self.current_action = action

        return {
            'new_temperature': new_temp,
            'action_taken': action,
            'reward': reward,
            'q_table_size': len(self.q_table),
            'exploration_rate': self.epsilon
        }

def simulate_ac_system():
    """
    Simulate the complete AC control system
    """
    print("=== AI AC Temperature Control System ===\n")

    # Initialize controllers
    print("1. Training Supervised Learning Model...")
    ml_controller = SmartACController()
    ml_controller.train_model()

    print("\n2. Initializing Reinforcement Learning Controller...")
    rl_controller = RLACController()

    print("\n3. Running Simulation...")
    print("-" * 60)

    # Simulate 24 hours of operation
    for hour in range(24):
        print(f"\nHour {hour:02d}:00")

        # Get current sensor data
        sensor_data = ml_controller.get_sensor_data()
        sensor_data['hour'] = hour

        # ML Model prediction
        ml_temp = ml_controller.predict_optimal_temperature(sensor_data)

        # RL Model decision
        rl_result = rl_controller.control_step(sensor_data)
        rl_temp = rl_result['new_temperature']

        # Display results
        print(f"Indoor: {sensor_data['indoor_temp']:.1f}°C | "
              f"Outdoor: {sensor_data['outdoor_temp']:.1f}°C | "
              f"Occupancy: {'Yes' if sensor_data['occupancy'] else 'No'}")
        print(f"ML Model suggests: {ml_temp}°C")
        print(f"RL Model suggests: {rl_temp}°C | "
              f"Reward: {rl_result['reward']:.2f}")

        # Simulate user occasionally adjusting temperature
        if random.random() < 0.1:  # 10% chance of user adjustment
            user_adjustment = ml_temp + random.gauss(0, 1)
            user_adjustment = max(20, min(28, user_adjustment))
            print(f"User adjusted to: {user_adjustment:.1f}°C")

            # Learn from user adjustment
            ml_controller.adaptive_learning(user_adjustment, sensor_data)

        # Small delay for simulation
        time.sleep(0.1)

    print(f"\nSimulation complete!")
    print(f"RL Agent learned {len(rl_controller.q_table)} state-action pairs")

    return ml_controller, rl_controller

def real_time_control_example():
    """
    Example of real-time AC control
    """
    print("\n=== Real-time Control Example ===")

    controller = SmartACController()
    controller.train_model()

    print("Starting real-time control loop...")
    print("(In real implementation, this would run continuously)")

    for i in range(5):
        print(f"\nControl cycle {i+1}:")

        # Run control cycle
        decision_log = controller.run_control_cycle()

        # Display decision
        sensor = decision_log['sensor_data']
        print(f"Current conditions: {sensor['indoor_temp']:.1f}°C, "
              f"Humidity: {sensor['humidity']:.0f}%, "
              f"Occupancy: {'Yes' if sensor['occupancy'] else 'No'}")
        print(f"AI Decision: Set AC to {decision_log['predicted_temp']}°C")
        print(f"Comfort Score: {decision_log['comfort_score']:.2f}")

        time.sleep(1)  # Wait 1 second between cycles

if __name__ == "__main__":
    # Run the complete demonstration
    ml_controller, rl_controller = simulate_ac_system()

    # Show real-time control example
    real_time_control_example()

    # Save the trained model
    ml_controller.save_model("smart_ac_model.pkl")

    print("\n=== System Ready ===")
    print("Your AI AC controller is trained and ready to use!")
    print("Key features:")
    print("- Learns from environmental conditions")
    print("- Adapts to user preferences")
    print("- Balances comfort and energy efficiency")
    print("- Provides real-time temperature control")

=== AI AC Temperature Control System ===

1. Training Supervised Learning Model...
Generating training data...
Training models...
random_forest - MSE: 1.286, MAE: 0.801
gradient_boost - MSE: 1.214, MAE: 0.795
Best model selected with MSE: 1.214

Feature Importance:
         feature  importance
0   outdoor_temp    0.563629
3      occupancy    0.421926
7        ac_diff    0.004954
5   energy_usage    0.002677
2       humidity    0.001546
1    indoor_temp    0.001282
12     month_sin    0.001171
6      temp_diff    0.001160

2. Initializing Reinforcement Learning Controller...

3. Running Simulation...
------------------------------------------------------------

Hour 00:00
Indoor: 14.7°C | Outdoor: 19.3°C | Occupancy: Yes
ML Model suggests: 18°C
RL Model suggests: 22°C | Reward: 3.90

Hour 01:00
Indoor: 12.1°C | Outdoor: 18.7°C | Occupancy: No
ML Model suggests: 18°C
RL Model suggests: 22°C | Reward: 0.80

Hour 02:00
Indoor: 17.4°C | Outdoor: 19.4°C | Occupancy: Yes
ML Model suggests: 19